This Notebook is mainly used for projects in the course of Applied Data Science Capstone

In [1]:
import pandas as pd
import numpy as np

In [2]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


# Peer-graded Assignment: Capstone Project - The Battle of Neighborhoods (Week 1)

# 1) Introduction/Business Problem : 
New York City's demographics show that it is a large and ethnically diverse metropolis. With it's diverse culture , comes diverse food items. There are many resturants in New york City, each beloning to different categories like Chinese , Indian , French etc.

So as part of this project , we will list and visualize all major parts of New York City that has great indian resturants.

# 2) Data :
For this project we need the following data:

1.New York City data that contains list Boroughs, Neighborhoods along with their latitude and longitude.

2.Indian resturants in each neighborhood of new york city.

3.GeoSpace data


Questions that will be answers at the end of this project :
1. What are best location in New York City for Indian Cuisine?
2. Which areas have potential Indian Restaurant Market?
3. Which all areas lack Indian Restaurants?
4. Which is the best place to stay if you prefer Indian Cuisine?

# Lets import necessary Libraries for our project

In [11]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests
from bs4 import BeautifulSoup
!pip install geocoder
import geocoder
import os
!pip install geopy
import folium # map rendering library
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
# Matplotlib and associated plotting modules
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
%matplotlib inline

Now we define a function to get the geocodes i.e latitude and longitude of a given location using geopy.

In [12]:
def geo_location(address):
    # get geo location of address
    geolocator = Nominatim(user_agent="ny_explorer")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    return latitude,longitude